In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
con = sqlite3.connect('db')

In [3]:
def select(sql):
    return pd.read_sql(sql, con)

In [4]:
sql = '''select * from german_credit'''
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,None,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,None,776,12,radio/TV,0,2007-07-21 13:22:14,358
...,...,...,...,...,...,...,...,...,...,...,...,...
995,65,male,2,free,little,little,2600,18,radio/TV,1,2007-12-16 20:17:19,624
996,30,male,3,own,little,moderate,4455,36,business,1,2007-07-12 14:08:58,181
997,33,male,2,own,little,moderate,6403,24,radio/TV,0,2008-04-08 03:24:26,730
998,29,female,2,own,None,None,5003,21,car,1,2007-11-29 15:51:45,557


In [7]:
! gdown --id 1ksjv4EsLItdn9rYA4oxislish-c4eE4D

Downloading...
From: https://drive.google.com/uc?id=1ksjv4EsLItdn9rYA4oxislish-c4eE4D
To: E:\Projects\glebmikha\course\german_credit_augmented_transactions.csv

  0%|          | 0.00/134k [00:00<?, ?B/s]
100%|##########| 134k/134k [00:00<00:00, 1.28MB/s]
100%|##########| 134k/134k [00:00<00:00, 1.27MB/s]


In [5]:
transactions = pd.read_csv('german_credit_augmented_transactions.csv')

In [6]:
transactions.head()

,dt,client_id,amount
0,2008-04-06 11:54:47,950,161.38
1,2007-07-28 00:00:19,418,35.34
2,2008-03-14 20:43:54,131,146.50
3,2007-12-18 13:03:24,353,119.21
4,2007-11-09 05:18:30,849,105.24


In [7]:
transactions.to_sql('client_transactions', con, if_exists='replace', index=False)

In [8]:
sql ='''select * from client_transactions t'''
select(sql)

,dt,client_id,amount
0,2008-04-06 11:54:47,950,161.38
1,2007-07-28 00:00:19,418,35.34
2,2008-03-14 20:43:54,131,146.50
3,2007-12-18 13:03:24,353,119.21
4,2007-11-09 05:18:30,849,105.24
...,...,...,...
4270,2007-08-18 04:05:05,185,10063.07
4271,2007-06-04 15:23:32,375,156.91
4272,2007-12-06 21:34:06,418,10053.82
4273,2008-04-19 17:30:07,409,10050.35


In [9]:
sql ='''select count (1) from client_transactions t'''
select(sql)

,count (1)
0,4275


In [10]:
sql ='''
select
     date(t.dt, 'start of month') as month,
     count(t.client_id) as trans_cnt,
     sum(t.amount) as amount_sum
from client_transactions t
group by 1
order by 1
'''
select(sql)

,month,trans_cnt,amount_sum
0,2007-05-01,338,450912.77
1,2007-06-01,379,551664.83
2,2007-07-01,304,494134.50
3,2007-08-01,255,426903.23
4,2007-10-01,332,634846.49
5,2007-11-01,389,500420.98
6,2007-12-01,364,561449.89
7,2008-01-01,413,630137.22
8,2008-02-01,228,337043.47
9,2008-03-01,309,425599.09


In [11]:
min ='''select min(date(t.dt, 'start of month')) from client_transactions t'''
max ='''select max(date(t.dt, 'start of month')) from client_transactions t'''

In [12]:
sql =f'''
WITH RECURSIVE dates(month) AS (
  VALUES(({min}))
  UNION ALL
  SELECT date(month, '+1 month')
  FROM dates
  WHERE month < ({max})
)
SELECT t.month FROM dates t;
'''
select(sql)

,month
0,2007-05-01
1,2007-06-01
2,2007-07-01
3,2007-08-01
4,2007-09-01
5,2007-10-01
6,2007-11-01
7,2007-12-01
8,2008-01-01
9,2008-02-01


In [13]:
sql ='''select min(date(t.dt, 'start of month')) from client_transactions t'''
select(sql)

,"min(date(t.dt, 'start of month'))"
0,2007-05-01


In [14]:
sql =f'''
WITH
dates(month) AS (
  VALUES(({min}))
  UNION ALL
  SELECT date(month, '+1 month')
  FROM dates
  WHERE month < ({max})
),

trans_month as
(
select
     date(t.dt, 'start of month') as month,
     count(t.client_id) as trans_cnt,
     sum(t.amount) as amount_sum
from client_transactions t
group by 1
order by 1
)


SELECT t.month,
 coalesce (tm.trans_cnt, 0) as trans_cnt,
 coalesce (tm.amount_sum, 0) as amount_sum
 FROM dates t
 left join trans_month tm on t.month=tm.month
 order by 1
'''
select(sql)

,month,trans_cnt,amount_sum
0,2007-05-01,338,450912.77
1,2007-06-01,379,551664.83
2,2007-07-01,304,494134.50
3,2007-08-01,255,426903.23
4,2007-09-01,0,0.00
5,2007-10-01,332,634846.49
6,2007-11-01,389,500420.98
7,2007-12-01,364,561449.89
8,2008-01-01,413,630137.22
9,2008-02-01,228,337043.47


In [17]:
sql ='''select distinct t.client_id from german_credit t'''
select(sql)

,client_id
0,210
1,929
2,200
3,45
4,358
...,...
995,624
996,181
997,730
998,557


In [32]:
sql ='''
select
     date(t.dt, 'start of month') as month,
     t.client_id,
     count(t.client_id) as trans_cnt,
     sum(t.amount) as amount_sum
from client_transactions t
group by 1, 2
order by 1
'''
select(sql)

,month,client_id,trans_cnt,amount_sum
0,2007-05-01,101,1,149.67
1,2007-05-01,107,2,217.50
2,2007-05-01,110,1,1081.32
3,2007-05-01,111,1,139.97
4,2007-05-01,113,1,1051.29
...,...,...,...,...
3551,2008-06-01,983,1,10050.37
3552,2008-06-01,987,1,192.48
3553,2008-06-01,992,1,-610.21
3554,2008-06-01,996,1,121.28


In [51]:
sql =f'''
WITH RECURSIVE dates(month) AS (
  VALUES(({min}))
  UNION ALL
  SELECT date(month, '+1 month')
  FROM dates
  WHERE month < ({max})
),

clients as (
    select distinct t.client_id from german_credit t
),

clients_month as (
    select t.month, c.client_id FROM dates t
    join clients c on 1=1
),

trans_month as (
    select
         date(t.dt, 'start of month') as month,
         t.client_id,
         count(t.client_id) as trans_cnt,
         sum(t.amount) as amount_sum
    from client_transactions t
    group by 1, 2
    order by 1
),

client_trans_month as(

select  t.*,
 tm.trans_cnt,
 tm.amount_sum,
 1 as user,
 case when tm.trans_cnt > 0 then 1 else 0 end as active

 from clients_month t
left join trans_month tm on t.client_id=tm.client_id and t.month=tm.month
--where t.client_id=929
)

select
t.month,
 sum(t.user) as user_cnt,
 sum(t.active) as active_cnt,
 sum(t.amount_sum) as amount_sum
  from client_trans_month t
  group by t.month
'''
t = select(sql)

In [53]:
t['amount_sum'].sum()

6548980.619999999

In [50]:
sql ='''select sum(t.amount) from client_transactions t'''
select(sql)

,sum(t.amount)
0,6548980.62
